# 4: Data Warehousing and On-line Analytical Processing

## What I can do:
- Data Warehousing
- On-line Analytical Processing (OLAP)

## 1. Introduction to Cubes

This chapter describes step-by-step how to use Cubes (http://cubes.databrewery.org/), a lightweight Python framework and set of tools for development of reporting and analytical applications, Online Analytical Processing (OLAP), multidimensional analysis and browsing of aggregated data. We will be working with v1.1 of Cubes. Cubes features:
- a logical view of analysed data - how analysts look at data, how they think of data, not not how the data are physically implemented in the data stores
- OLAP and aggregated browsing (default backend is for relational databse - ROLAP)
- hierarchical dimensions (attributes that have hierarchical dependencies, such as category-subcategory or country-region)
- multiple hierarchies in a dimension
- localizable metadata and data (see Localization)
- authentication and authorization of cubes and their data
- pluggable data warehouse – plug-in other cube-like (multidimensional) data sources

Cubes is meant to be used by application builders that want to provide analytical functionality. Cubes also relies on methods from SQLAlchemy (https://www.sqlalchemy.org/), an open-source SQL toolkit and object-relational mapper for Python.

## 2. Data Preparation

The example data used here is the International Bank for Reconstruction and Development (IBRD) Balance Sheet. For this example we will be using the CSV file "IBRD_Balance_Sheet__FY2010.csv" which is provided in the supplementary material for the lab. The CSV file includes records which are characterised by a Category (and subcategories), Line Item, Fiscal Year, and Amount (in US$ millions). We first start with imports:

In [1]:
from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv

We can now load the data, create a table and populate it with contents of the CSV file. Note the categories and subcategories created; check the CSV file to link the below commands with the contents of the dataset.

In [2]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                     )

# create a table, and assign a name (ibrd_balance)
# fields: our priginal dataset has the labels, here we just assign the type for them

## 3. Creating a data cube

Everything in Cubes happens in an *analytical workspace*. It contains cubes, maintains connections to the data stores (with cube data), provides connection to external cubes and more.

The workspace properties are specified in a configuration file slicer.ini (default name). The first thing we have to do is to specify a data store – a database which will host the cube’s data:

In [3]:
from cubes import Workspace

workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")

The structure of data cubes (in terms of dimensions, measures, and aggregates) is specified in JSON files. We now import file 'tutorial_model.json' (found in the lab supplementary material) which includes an example model of the data cube, dimension tables, and aggregate functions for the CSV file we loaded previously.

In [4]:
workspace.import_model("tutorial_model.json")

**Please make sure to inspect the structure of the above JSON file - this will be relevant for one of the assignment questions.**

We can now create a data cube based on the above data cube model and data table:

In [5]:
cube = workspace.cube("ibrd_balance")
# this is from the csv

## 4. Aggregations and OLAP operations

*Browser* is an object that does the actual aggregations and other data queries for a cube. To obtain one:

In [6]:
browser = workspace.browser(cube)

We can now compute aggregates of the data cube as specified by the data cube model. For computing the total count of records:

In [7]:
#len(row)

result = browser.aggregate()
result.summary["record_count"]

62

For computing a sum of the amount:

In [8]:
# from json, aggreates: name: amount_sum, function: sum, measure (data source): amount


result.summary["amount_sum"]

1116860

Now we can try to compute aggregates by year:

In [9]:
# QUESTION 3
result = browser.aggregate(drilldown=["year"])
for record in result:
    print(record)


{'year': 2009, 'amount_sum': 550840, 'amount_min': -1683, 'amount_max': 110040, 'record_count': 31}
{'year': 2010, 'amount_sum': 566020, 'amount_min': -3043, 'amount_max': 128577, 'record_count': 31}


Or compute aggregates by item category:

In [10]:
result = browser.aggregate(drilldown=["item"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'amount_sum': 558430, 'amount_min': 0, 'amount_max': 118104, 'record_count': 32}
{'item.category': 'e', 'item.category_label': 'Equity', 'amount_sum': 77592, 'amount_min': -3043, 'amount_max': 29870, 'record_count': 8}
{'item.category': 'l', 'item.category_label': 'Liabilities', 'amount_sum': 480838, 'amount_min': 8, 'amount_max': 128577, 'record_count': 22}


We can also perform *slicing* and *dicing* operations on the data cube. The below example performs a slicing operation on the data cube by selecting only entries with the year being 2009, and displays aggregates according to the item category. Here, a *cell* defines a point of interest – portion of the cube to be aggergated or browsed.

In [11]:
import cubes as cubes
cuts = [cubes.PointCut("year", ["2009"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["item"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'amount_sum': 275420, 'amount_min': 33, 'amount_max': 103657, 'record_count': 16}
{'item.category': 'e', 'item.category_label': 'Equity', 'amount_sum': 40037, 'amount_min': -1683, 'amount_max': 29870, 'record_count': 4}
{'item.category': 'l', 'item.category_label': 'Liabilities', 'amount_sum': 235383, 'amount_min': 57, 'amount_max': 110040, 'record_count': 11}


It's worth noting that in Cubes, slicing operations can be created by either specifying a "point cut" which selects a single value of an attribute in a given dimension (called using the cubes.PointCut() function as above), or by specifying a "range cut", which selects a range of values for a given dimension. The range cut can be called using the cubes.RangeCut() function, which takes as input the attribute name, the minimum value of the specified range, and the maximum value of the range.

Similarly, we can perform a *dicing* operation on the data cube by performing a selection on two or more dimensions. The below example performs a dicing operation on the data cube, selecting entries with the year being 2009 and the item category being "a", and displays the aggregate results:

In [12]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item"])
result.summary

{'amount_sum': 275420,
 'amount_min': 33,
 'amount_max': 103657,
 'record_count': 16}

We can also *drill down* lower in the Category hierarchy. Here, we perform a dicing operation to select records with year being 2009 and item category being "a" (corresponding to assets), and show aggregates for each subcategory level.

In [13]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item:subcategory"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'da', 'item.subcategory_label': 'Derivative Assets', 'amount_sum': 123065, 'amount_min': 2246, 'amount_max': 82793, 'record_count': 4}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'dfb', 'item.subcategory_label': 'Due from Banks', 'amount_sum': 3044, 'amount_min': 664, 'amount_max': 2380, 'record_count': 2}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'i', 'item.subcategory_label': 'Investments', 'amount_sum': 41012, 'amount_min': 41012, 'amount_max': 41012, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'lo', 'item.subcategory_label': 'Loans Outstanding', 'amount_sum': 103657, 'amount_min': 103657, 'amount_max': 103657, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'nn', 'item.subcategory_label': 'Nonnegotiable', 'amount_sum': 1202, 'amount_min': 1202, '

1. We can use the same CSV file and data cube in the above tutorial, modify the "tutorial_model.json" file to include aggregate measures for the minimum and maximum amount per year in the data cube.

- JSON File:
```
{
    "dimensions": [
        {
         "name":"item",
         "levels": [
                {
                    "name":"category",
                    "label":"Category",
                    "attributes": ["category", "category_label"]
                },
                {
                    "name":"subcategory",
                    "label":"Sub-category",
                    "attributes": ["subcategory", "subcategory_label"]
                },
                {
                    "name":"line_item",
                    "label":"Line Item",
                    "attributes": ["line_item"]
                }
            ]
        },
        {"name":"year", "role": "time"}
    ],
    "cubes": [
        {
            "name": "ibrd_balance",
            "dimensions": ["item", "year"],
            "measures": [{"name":"amount", "label":"Amount"}],
            "aggregates": [
                    {
                        "name": "amount_sum",
                        "function": "sum",
                        "measure": "amount"
                    }, 
                    {
                        "name": "amount_min",
                        "function": "min",
                        "measure": "amount"
                    },
                    {
                        "name": "amount_max",
                        "function": "max",
                        "measure": "amount"
                    },
                    {
                        "name": "record_count",
                        "function": "count"
                    }
                ],
            "mappings": {
                          "item.line_item": "line_item",
                          "item.subcategory": "subcategory",
                          "item.subcategory_label": "subcategory_label",
                          "item.category": "category",
                          "item.category_label": "category_label"
                         },
            "info": {
                "min_date": "2010-01-01",
                "max_date": "2010-12-31"
            }
        }
    ]
}

```



> Using the same CSV file and data cube in the above lab tutorial, modify the "tutorial_model.json" file to 
- include aggregate measures for the minimum and maximum amount in the data cube. 

In [14]:
result = browser.aggregate()
print (result.summary)

{'amount_sum': 1116860, 'amount_min': -3043, 'amount_max': 128577, 'record_count': 62}


- Using these implemented aggregate measures, produce the values for the minimum and maximum amount in the data per year.

In [15]:
result = browser.aggregate(drilldown=["year"])
for record in result:
    print(record)

{'year': 2009, 'amount_sum': 550840, 'amount_min': -1683, 'amount_max': 110040, 'record_count': 31}
{'year': 2010, 'amount_sum': 566020, 'amount_min': -3043, 'amount_max': 128577, 'record_count': 31}


2. We can also use the CSV file "country-income.csv", perform the following:
  1. Load the CSV file using Cubes, create a JSON file for the data cube model, and create a data cube for the data. Use as dimensions the region, age, and online shopper fields. Use as measure the income. Define aggregate functions in the data cube model for the total, average, minimum, and maximum income.
  2. Using the created data cube and data cube model, produce aggregate results for: the whole data cube; results per region; results per online shopping activity; and results for all people aged between 40 and 50. 

- JSON File:

```{
    "dimensions": [{"name":"Region"},
                {"name":"Age"},
                {"name":"Online_shopper"}],
    "cubes": [
        {
            "name": "country_income",
            "dimensions": ["Region", "Age", "Online_shopper"],
            "measures": [{"name":"Income"}],
            "aggregates": [
                    {
                        "name": "amount_total",
                        "function": "sum",
                        "measure": "Income"
                    }, 
                    {
                        "name": "amount_average",
                        "function": "avg",
                        "measure": "Income"
                    },
                    {
                        "name": "amount_min",
                        "function": "min",
                        "measure": "Income"
                    },
                    {
                        "name": "amount_max",
                        "function": "max",
                        "measure": "Income"
                    }
                ],
            "mappings": {},
            "info": {}
    }
    ]
}
```

> Load the CSV file using Cubes, create a JSON file for the data cube model, and create a data cube for the data.
1. Use as dimensions the region, age, and online shopper fields. 
2. Use as measure the income. 

In [19]:
create_table_from_csv(engine,
                      "country-income.csv",
                      table_name="country_income",
                      fields=[
                          ("Region", "string"),
                          ("Age", "integer"),
                          ("Income", "integer"),
                          ("Online_shopper", "boolean")],
                      create_id=True
                     )

workspace.import_model("income_model.json")
cube = workspace.cube("country_income")
browser = workspace.browser(cube)

> Using the created data cube and data cube model:  
1. Define aggregate functions in the data cube model for the total, average, minimum, and maximum income. 
2. Using the created data cube and data cube model, produce aggregate results for: 

In [21]:
# the whole data cube;
result = browser.aggregate()
print (result.summary)

{'amount_total': 768200, 'amount_average': 76820.0, 'amount_min': 57600, 'amount_max': 99600}


In [22]:
# results per region; 
result = browser.aggregate(drilldown=["Region"])
for record in result:
    print(record)

{'Region': 'Brazil', 'amount_total': 193200, 'amount_average': 64400.0, 'amount_min': 57600, 'amount_max': 73200}
{'Region': 'India', 'amount_total': 331200, 'amount_average': 82800.0, 'amount_min': 69600, 'amount_max': 94800}
{'Region': 'USA', 'amount_total': 243800, 'amount_average': 81266.66666666667, 'amount_min': 64800, 'amount_max': 99600}


In [23]:
# results per online shopping activity; 
result = browser.aggregate(drilldown=["Online_shopper"])
for record in result:
    print(record)

{'Online_shopper': 'No', 'amount_total': 386400, 'amount_average': 77280.0, 'amount_min': 62400, 'amount_max': 99600}
{'Online_shopper': 'Yes', 'amount_total': 381800, 'amount_average': 76360.0, 'amount_min': 57600, 'amount_max': 94800}


In [24]:
# results for all people aged between 40 and 50.
import cubes as cubes

cuts = [
    cubes.RangeCut("Age",[40], [50])
]

cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["Age"])
    
print(result.summary)

{'amount_total': 451400, 'amount_average': 75233.33333333333, 'amount_min': 62400, 'amount_max': 86400}
